In [ ]:
## 新的染色体互作矩阵
import pandas as pd
import sys
matrix_f = sys.argv[1]
bed_f    = sys.argv[2]
output_f = sys.argv[3]
df_mtx = pd.read_csv(matrix_f, sep="\t", names=['bin1', 'bin2', 'interaction'])
df_bed = pd.read_csv(bed_f, sep="\t", names=['chr1', 'start1', 'end1', 'bin1'])
df1 = pd.merge(df_mtx, df_bed, how='left', on=['bin1'])
df_bed.columns = ['chr2', 'start2', 'end2', 'bin2']
df2 = pd.merge(df_mtx, df_bed, how='left', on=['bin2'])
df2.to_csv(output_f, sep='\t', columns = ['chr1', 'start1', 'end1', 'chr2', 'start2', 'end2', 'interaction'])

In [ ]:
## 染色体间互作矩阵
import pandas as pd
chromosomes = ['Ghir_A01', 'Ghir_A02', 'Ghir_A03', 'Ghir_A04', 'Ghir_A05', 'Ghir_A06', 'Ghir_A07', 'Ghir_A08', 'Ghir_A09', 'Ghir_A10', 'Ghir_A11', 'Ghir_A12', 'Ghir_A13', 'Ghir_D01', 'Ghir_D02', 'Ghir_D03', 'Ghir_D04', 'Ghir_D05', 'Ghir_D06', 'Ghir_D07', 'Ghir_D08', 'Ghir_D09', 'Ghir_D10', 'Ghir_D11', 'Ghir_D12', 'Ghir_D13']
for i in range(26):
    k = i+1
    for j in range(k, 26):
        chr1, chr2 = chromosomes[i], chromosomes[j]
        one_mtx, two_mtx = "{}.matrix".format(chr1), "{}.matrix".format(chr2)
        one_abs, two_abs = "{}_abs.bed".format(chr1), "{}_abs.bed".format(chr2)
        one_biases, two_biases = "{}.biases".format(chr1), "{}.biases".format(chr2)
        out_mtx, out_abs, out_biases = "inter/{0}_{1}.matrix".format(chr1, chr2), "inter/{0}_{1}_abs.bed".format(chr1, chr2), "inter/{0}_{1}.biases".format(chr1, chr2)
        df_mtx1 = pd.read_csv(one_mtx, sep="\t", names=['bin1', 'bin2', 'reads'])
        df_mtx2 = pd.read_csv(two_mtx, sep="\t", names=['bin1', 'bin2', 'reads'])
        df_abs1 = pd.read_csv(one_abs, sep="\t", names=['chr', 'start', 'end', 'tags'])
        df_abs2 = pd.read_csv(two_abs, sep="\t", names=['chr', 'start', 'end', 'tags'])
        df_biases1 = pd.read_csv(one_biases, sep="\t", names=['biases'])
        df_biases2 = pd.read_csv(two_biases, sep="\t", names=['biases'])
        ## second matrix format
        start = df_abs1.shape[0]+1
        end = start + df_abs2.shape[0]
        df_abs2['tags2'] = range(start, end)
        df_abs2_tags = df_abs2[['tags', 'tags2']].copy()
        df_abs2_tags.columns = ['bin1', 'tags1']
        df1 = pd.merge(df_mtx2, df_abs2_tags, how='inner', on=['bin1'])
        df_abs2_tags.columns = ['bin2', 'tags2']
        df2 = pd.merge(df1, df_abs2_tags, how='inner', on=['bin2'])
        df_mtx2 = df2[['tags1', 'tags2', 'reads']]
        df_mtx2.columns = ['bin1', 'bin2', 'reads']
        df_abs2 = df_abs2[['chr', 'start', 'end', 'tags2']]
        df_abs2.columns = ['chr', 'start', 'end', 'tags']
        # concat file
        df_mtx = pd.concat(df_mtx1, df_mtx2, axis=0)
        df_abs = pd.concat(df_abs1, df_abs2, axis=0)
        df_biases = pd.concat(df_biases1, df_biases2, axis=0)
        #write file
        df_mtx.to_csv(out_mtx, sep='\t', header=False, index=False)
        df_abs.to_csv(out_abs, sep='\t', header=False, index=False)
        df_biases.to_csv(out_biases, sep='\t', header=False, index=False) 

In [ ]:
## 用GenomeDISCO， HiCRep， HiC-Spector， QuASAR-Rep 来检测数据的质量
import pandas as pd
import os
import sys
tissue = ['anthers1', 'cotyledon1', 'ovules1',  'hypocotyl1', 'leaf1', 'petals1', 'radicle1', 'root1', 'stem1', 'stigma1', 'fiber_5DPA1', 'fiber_10DPA1', 'fiber_20DPA1', 'anthers2', 'cotyledon2', 'ovules2',  'hypocotyl2', 'leaf2', 'petals2', 'radicle2', 'root2', 'stem2', 'stigma2', 'fiber_5DPA2', 'fiber_10DPA2', 'fiber_20DPA2']
#rep1 = ['anthers1', 'cotyledon1', 'ovules1',  'hypocotyl1', 'leaf1', 'petals1', 'radicle1', 'root1', 'stem1', 'stigma1', 'fiber_5DPA1', 'fiber_10DPA1', 'fiber_20DPA1']
#rep2 = ['anthers2', 'cotyledon2', 'ovules2',  'hypocotyl2', 'leaf2', 'petals2', 'radicle2', 'root2', 'stem2', 'stigma2', 'fiber_5DPA2', 'fiber_10DPA2', 'fiber_20DPA2']
output_f = sys.argv[1] 
df_result = pd.DataFrame(columns=tissue, index=tissue)
for i in tissue: 
    for j in tissue:
        input_f = "{0}.vs.{1}.txt".format(i,j)
        if os.path.exists(input_f):
            df_tmp = pd.read_csv(input_f, sep='\t', names=['tags1', 'tags2', 'chrs', 'value'])
            aver_value = df_tmp['value'].mean()
            df_result.loc[i, j] = aver_value
df_result.to_csv(output_f, sep='\t')